In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls /content/drive/MyDrive/4sq/input/

lgb_012.lgb	       train.csv	   train_data_1_1.zip  train_data3.csv
lgb_01.lgb	       train_data_0_1.csv  train_data_1_2.csv  train_data4.csv
lgb_02.lgb	       train_data_0_2.csv  train_data_1_3.csv  train_data5.csv
lgb_12.lgb	       train_data_0_3.csv  train_data_1_4.csv  tv_ids_d.npy
pairs.csv	       train_data_0_4.csv  train_data_1_5.csv
sample_submission.csv  train_data_0_5.csv  train_data1.csv
test.csv	       train_data_1_1.csv  train_data2.csv


In [4]:
!python -m pip install "dask[complete]"
! pip3 install optuna
! pip3 install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
## Imports
import warnings
warnings.filterwarnings('ignore')

import os
import gc
import time
import random
import difflib
import multiprocessing
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm.auto import tqdm
from sklearn.model_selection import GroupKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

import dask.dataframe as dd

In [6]:
OPTUNA_FLAG = False

In [7]:
input_type = {
'id':                    'object',
'match_id':              'object',
'kdist':                 'float32',
'kneighbors':            'float32',
'kdist_country':         'float32',
'latdiff':                 'float32',
'londiff':                 'float32',
'manhattan':            'float32',
'euclidean':              'float32',
'haversine':              'float32',
'kneighbors_country':    'float32',
'label':                 'bool',
'name_sim':              'float32',
'name_gesh':             'float32',
'name_leven':            'float32',
'name_jaro':             'float32',
'name_lcs':              'float32',
'name_len_diff':         'uint8',
'name_nleven':           'float32',
'name_nlcsk':            'float32',
'name_nlcs':             'float32',
'address_sim':           'float32',
'address_gesh':          'float32',
'address_leven':         'float32',
'address_jaro':          'float32',
'address_lcs':           'float32',
'address_len_diff':      'uint8',
'address_nleven':        'float32',
'address_nlcsk':         'float32',
'address_nlcs':          'float32',
'city_gesh':             'float32',
'city_leven':            'float32',
'city_jaro':             'float32',
'city_lcs':              'float32',
'city_len_diff':         'uint8',
'city_nleven':           'float32',
'city_nlcsk':            'float32',
'city_nlcs':             'float32',
'state_sim':             'float32',
'state_gesh':            'float32',
'state_leven':           'float32',
'state_jaro':            'float32',
'state_lcs':             'float32',
'state_len_diff':        'uint8',
'state_nleven':          'float32',
'state_nlcsk':           'float32',
'state_nlcs':            'float32',
'zip_gesh':              'float32',
'zip_leven':             'float32',
'zip_jaro':              'float32',
'zip_lcs':               'float32',
'url_sim':               'float32',
'url_gesh':              'float32',
'url_leven':             'float32',
'url_jaro':              'float32',
'url_lcs':               'float32',
'url_len_diff':          'uint8',
'url_nleven':            'float32',
'url_nlcsk':             'float32',
'url_nlcs':              'float32',
'phone_gesh':            'float32',
'phone_leven':           'float32',
'phone_jaro':            'float32',
'phone_lcs':             'float32',
'categories_sim':        'float32',
'categories_gesh':       'float32',
'categories_leven':      'float32',
'categories_jaro':       'float32',
'categories_lcs':        'float32',
'categories_len_diff':   'uint8',
'categories_nleven':     'float32',
'categories_nlcsk':      'float32',
'categories_nlcs':       'float32',
'country_sim':           'float32',
'country_gesh':          'float32',
'country_leven':         'float32',
'country_jaro':          'float32',
'country_lcs':           'float32',
'country_len_diff':      'bool',
'country_nleven':        'float32',
'country_nlcsk':         'float32',
'country_nlcs':          'float32',
'kdist_diff':            'float32',
'kneighbors_mean':       'float32',
'country_x':             'object',
'country_y':             'object',
'latitude_x':            'float32',
'latitude_y':            'float32',
'longitude_x':            'float32',
'longitude_y':            'float32',
}
cat_list = ['country_x', 'country_y']

In [8]:
!ls drive/MyDrive/4sq/input/train_data1.csv

drive/MyDrive/4sq/input/train_data1.csv


In [9]:
#train_list = [dd.read_csv(f'drive/MyDrive/4sq/input/train_data_0_{i}.csv', dtype=input_type).compute() for i in range(1,6)]
train_list = [dd.read_csv(f'drive/MyDrive/4sq/input/train_data_1_{i}.csv', dtype=input_type).compute() for i in range(1,6)]

In [10]:
# 0,1,2,3,4
cv = 0
#train = pd.concat([train_list[cv], train_list[(cv+1)%5], train_list[(cv+2)%5], train_list[(cv+3)%5], train_list[(cv+4)%5]], ignore_index=True)
train = pd.concat([train_list[0], train_list[1], train_list[2], train_list[3], train_list[4]], ignore_index=True)

#valid = dd.read_csv('drive/MyDrive/4sq/input/train_data_1_1.csv', dtype=input_type).compute() 
valid = dd.read_csv('drive/MyDrive/4sq/input/train_data_0_1.csv', dtype=input_type).compute() 

del train_list
gc.collect()

0

In [11]:
# importanceの低い素性を削る
train = train.drop(['country_jaro', 'country_nlcsk', 'country_lcs', 'country_len_diff', 'country_nlcs', 'country_nleven', 'country_leven'], axis=1)
valid = valid.drop(['country_jaro', 'country_nlcsk', 'country_lcs', 'country_len_diff', 'country_nlcs', 'country_nleven', 'country_leven'], axis=1)

In [12]:
# 改めてカテゴリに変換
train['country_x'] = train['country_x'].fillna('nanstr')
train['country_y'] = train['country_y'].fillna('nanstr')
train['country_x'] = train['country_x'].astype('category')
train['country_y'] = train['country_y'].astype('category')

valid['country_x'] = valid['country_x'].fillna('nanstr')
valid['country_y'] = valid['country_y'].fillna('nanstr')
valid['country_x'] = valid['country_x'].astype('category')
valid['country_y'] = valid['country_y'].astype('category')

In [13]:
# 自分自身とのmatchは学習には不要なので除く
train = train.query('id != match_id')
valid = valid.query('id != match_id')

In [14]:
train

,id,match_id,kdist,kneighbors,kdist_country,kneighbors_country,label,latdiff,londiff,manhattan,...,categories_jaro,categories_lcs,categories_len_diff,categories_nleven,categories_nlcsk,categories_nlcs,country_sim,country_gesh,kdist_diff,kneighbors_mean
21,E_000260dd24cecf,E_919a37430bea7a,0.0,0.0,0.0,1.0,True,0.000000,0.000000,0.000000,...,1.000000,15.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,0.5
140,E_000fa08832fbd7,E_0146e712d66bb8,0.0,0.0,0.0,15.0,False,0.000000,0.000000,0.000000,...,1.000000,21.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,7.5
282,E_00219328bc7ecb,E_6f272b9533b508,0.0,0.0,0.0,0.0,True,0.000000,0.000000,0.000000,...,1.000000,12.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,0.0
308,E_0024fc2bd72ad5,E_b2cc39e9379c00,0.0,0.0,0.0,1.0,True,0.000000,0.000000,0.000000,...,NaN,NaN,0,NaN,NaN,NaN,1.0,1.0,NaN,0.5
682,E_005143e7c63ed9,E_bc96ffb9eccdb4,0.0,0.0,0.0,0.0,False,0.000000,0.000000,0.000000,...,0.538047,7.0,12,0.766667,0.233333,0.388889,1.0,1.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18352833,E_ffebf327be8677,E_fe3b25a6be0358,NaN,NaN,NaN,NaN,False,0.036726,0.386166,0.422892,...,1.000000,12.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,NaN
18352834,E_ffebf327be8677,E_f850acfc89e173,NaN,NaN,NaN,NaN,False,0.132271,0.047146,0.179417,...,1.000000,12.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,NaN
18352835,E_fff368a2c59546,E_de2bdda6d4977c,NaN,NaN,NaN,NaN,False,0.000085,0.001298,0.001383,...,1.000000,31.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,NaN
18352836,E_fff9ecf69690da,E_393d42bc0cdbce,NaN,NaN,NaN,NaN,True,0.000008,0.000011,0.000018,...,1.000000,5.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,NaN


In [15]:
valid

,id,match_id,kdist,kneighbors,kdist_country,kneighbors_country,label,latdiff,londiff,manhattan,...,categories_jaro,categories_lcs,categories_len_diff,categories_nleven,categories_nlcsk,categories_nlcs,country_sim,country_gesh,kdist_diff,kneighbors_mean
26,E_00045e5c3e3ed6,E_18f8dc860a9f97,0.0,0.0,0.0,0.0,True,0.000000,0.000000,0.000000,...,0.836364,6.0,27,0.818182,1.000000,0.181818,1.0,1.0,NaN,0.0
39,E_0005d7b831dd07,E_e614e493a785aa,0.0,0.0,0.0,0.0,False,0.000000,0.000000,0.000000,...,0.591663,8.0,23,0.783784,0.216216,0.571429,1.0,1.0,NaN,0.0
185,E_00180d302af492,E_13a836ee1dd8ff,0.0,0.0,0.0,0.0,True,0.000000,0.000000,0.000000,...,1.000000,18.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,0.0
261,E_0020ff8f3b06f9,E_05aff53ddd7f6c,0.0,0.0,NaN,NaN,False,0.000000,0.000000,0.000000,...,1.000000,21.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,0.0
351,E_002c767fde5d45,E_ab9e15c9d98f49,0.0,0.0,0.0,3.0,False,0.000000,0.000000,0.000000,...,0.522222,4.0,6,0.800000,0.266667,0.444444,1.0,1.0,NaN,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63720,E_3319b6a51b523e,E_b051830fe6713b,NaN,NaN,NaN,NaN,False,0.000011,0.000569,0.000579,...,1.000000,9.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,NaN
63721,E_3319b6a51b523e,E_2bd91454bcd76b,NaN,NaN,NaN,NaN,False,0.000011,0.000568,0.000579,...,1.000000,9.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,NaN
63722,E_3319b6a51b523e,E_c92f79dcfa7764,NaN,NaN,NaN,NaN,False,0.000011,0.000568,0.000579,...,1.000000,9.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,NaN
63723,E_331aa34db9bd39,E_6e2fd5abbbc52e,NaN,NaN,NaN,NaN,True,0.001186,0.000134,0.001320,...,0.561869,7.0,13,0.708333,0.636364,0.291667,1.0,1.0,NaN,NaN


In [16]:
train.columns.values

array(['id', 'match_id', 'kdist', 'kneighbors', 'kdist_country',
       'kneighbors_country', 'label', 'latdiff', 'londiff', 'manhattan',
       'euclidean', 'haversine', 'latitude_x', 'latitude_y',
       'longitude_x', 'longitude_y', 'name_sim', 'name_gesh',
       'name_leven', 'name_jaro', 'name_lcs', 'name_len_diff',
       'name_nleven', 'name_nlcsk', 'name_nlcs', 'country_x', 'country_y',
       'address_sim', 'address_gesh', 'address_leven', 'address_jaro',
       'address_lcs', 'address_len_diff', 'address_nleven',
       'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven',
       'city_jaro', 'city_lcs', 'city_len_diff', 'city_nleven',
       'city_nlcsk', 'city_nlcs', 'state_sim', 'state_gesh',
       'state_leven', 'state_jaro', 'state_lcs', 'state_len_diff',
       'state_nleven', 'state_nlcsk', 'state_nlcs', 'zip_gesh',
       'zip_leven', 'zip_jaro', 'zip_lcs', 'url_sim', 'url_gesh',
       'url_leven', 'url_jaro', 'url_lcs', 'url_len_diff', 'url_nleven',
       '

In [17]:
# 学習データを特徴量と目的変数に分ける
train_x = train.drop(['label'], axis=1)
train_y = train['label']

valid_x = valid.drop(['label'], axis=1)
valid_y = valid['label']

In [18]:
# 特徴量作成
train_x = train_x.drop(['id', 'match_id'], axis=1)
valid_x = valid_x.drop(['id', 'match_id'], axis=1)

In [19]:
def reduce_mem_usage(df, verbose=True):
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2
  
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)
          else:
              if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                  df[col] = df[col].astype(np.float16)
              elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)
  
  end_mem = df.memory_usage().sum() / 1024**2
  print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
  print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
  
  return df

#train_x = reduce_mem_usage(train_x)
#valid_x = reduce_mem_usage(valid_x)

In [20]:
categorical_features_indices = np.where(train_x.dtypes == 'category')[0]
categorical_features_indices

array([22, 23])

### Training

In [21]:
from catboost import Pool
import optuna
from catboost import CatBoostClassifier, Pool, MetricVisualizer
from sklearn import metrics

In [22]:
train_pool = Pool(train_x, train_y, cat_features=categorical_features_indices)
valid_pool = Pool(valid_x, valid_y, cat_features=categorical_features_indices)

In [23]:
feature_names = train_x.columns

In [24]:
# パラメータの指定, optuna result
"""
params = {
    'loss_function' : 'Logloss',
    'eval_metric' : 'AUC',
    'iterations' : 1000,                         
    'depth' : 10,                                       
    'learning_rate' : 0.15370,               
    'random_strength' : 18,                       
    'bagging_temperature' : 0.05390, 
    'od_wait' :17,
    'od_type': 'Iter',
    'od_wait': 28,
    'random_seed': 0,
    'task_type': 'GPU',
    }
"""

params = {
    'loss_function' : 'Logloss',
    'eval_metric' : 'AUC',
    'iterations' : 1000,                         
    'depth' : 10,                                       
    #'learning_rate' : 0.2987036544125214,  
    'learning_rate' : 0.1,             
    'random_strength' : 1,                       
    'bagging_temperature' : 0.10721404311739535, 
    'od_wait' :17,
    'od_type': 'IncToDec',
    'od_wait': 28,
    'random_seed': 0,
    'task_type': 'GPU',
}

In [ ]:
def objective(trial):
    # パラメータの指定
    params = {
        'loss_function' : 'Logloss',
        'eval_metric' : 'AUC',
        'iterations' : trial.suggest_int('iterations', 50, 1000),                         
        'depth' : trial.suggest_int('depth', 4, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),               
        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), 
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait' :trial.suggest_int('od_wait', 10, 50),
        'task_type': 'GPU',
    }

    model = CatBoostClassifier(**params)
    model.fit(train_pool, eval_set=valid_pool, plot=True, use_best_model = True, early_stopping_rounds=20,)
    # どっち使ってもいいはず
    #predictions = model.predict_proba(valid_pool)[:,1]
    #auc = metrics.roc_auc_score(valid_y, predictions)
    #return 1.0 - auc
    return 1.0 - model.get_best_score()['validation']['AUC']

In [25]:
# optunaで最適値を見つける
if OPTUNA_FLAG:
  study = optuna.create_study()
  study.optimize(objective, n_trials=100)

In [25]:
model = CatBoostClassifier(**params)

In [26]:
model.fit(train_pool, 
          eval_set=valid_pool, 
          plot=True,
          early_stopping_rounds=20,
          )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9672313	best: 0.9672313 (0)	total: 778ms	remaining: 12m 56s
1:	total: 1.55s	remaining: 12m 54s
2:	total: 2.39s	remaining: 13m 14s
3:	total: 3.22s	remaining: 13m 22s
4:	total: 3.99s	remaining: 13m 14s
5:	test: 0.9720783	best: 0.9722893 (4)	total: 4.83s	remaining: 13m 20s
6:	total: 5.59s	remaining: 13m 13s
7:	total: 6.45s	remaining: 13m 19s
8:	total: 7.28s	remaining: 13m 21s
9:	total: 8.13s	remaining: 13m 24s
10:	test: 0.9769391	best: 0.9769391 (10)	total: 8.99s	remaining: 13m 28s
11:	total: 9.76s	remaining: 13m 23s
12:	total: 10.5s	remaining: 13m 20s
13:	total: 11.3s	remaining: 13m 15s
14:	total: 12.1s	remaining: 13m 17s
15:	test: 0.9796677	best: 0.9796677 (15)	total: 12.9s	remaining: 13m 16s
16:	total: 13.7s	remaining: 13m 12s
17:	total: 14.4s	remaining: 13m 6s
18:	total: 15.1s	remaining: 12m 58s
19:	total: 15.9s	remaining: 12m 59s
20:	test: 0.9811087	best: 0.9811087 (20)	total: 16.7s	remaining: 12m 59s
21:	total: 17.5s	remaining: 12m 56s
22:	total: 18.2s	remaining: 12m 53s


In [27]:
model.get_params()

{'bagging_temperature': 0.10721404311739535,
 'depth': 10,
 'eval_metric': 'AUC',
 'iterations': 1000,
 'learning_rate': 0.1,
 'loss_function': 'Logloss',
 'od_type': 'IncToDec',
 'od_wait': 28,
 'random_seed': 0,
 'random_strength': 1,
 'task_type': 'GPU'}

In [28]:
model.get_best_iteration()

998

In [29]:
model.get_best_score()['validation']['AUC']

0.9888603091239929

In [30]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = train_x.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

londiff: 10.144717332127643
address_nleven: 6.428996485234373
latitude_y: 4.236735922683714
latitude_x: 4.21509152124795
name_sim: 3.719855291818871
categories_sim: 3.405769409523827
longitude_y: 3.310428647299633
longitude_x: 3.073012500482149
kneighbors_mean: 2.9161267679500247
categories_lcs: 2.9033946210427857
country_y: 2.8531393441752706
manhattan: 2.7767102904577956
country_x: 2.701666115522312
categories_nleven: 2.516218752067017
phone_leven: 2.318888875079042
categories_jaro: 1.9961566715406245
name_gesh: 1.8883658935980645
name_lcs: 1.7386291328897476
haversine: 1.4945611534322119
categories_len_diff: 1.4855484473879994
name_jaro: 1.4723536120419192
address_sim: 1.453820065883238
categories_gesh: 1.4290887280356157
address_leven: 1.4282143894191304
kdist_country: 1.3768237329497788
kdist: 1.2660111771408327
city_len_diff: 1.2058728242914678
kneighbors: 1.1758612110864493
categories_leven: 1.1630537912693582
latdiff: 1.1200865967313158
name_leven: 1.0658293116060358
euclidean:

In [31]:
# save model
import pickle
# save model
#pickle.dump(model, open(f'catboost_0.pkl', 'wb'))
pickle.dump(model, open(f'catboost_1.pkl', 'wb'))

### Check
- メトリクスの目安となる値を求める

In [32]:
pred = model.predict_proba(valid_pool)[:,1]

In [33]:
metrics.roc_auc_score(valid_y, pred)

0.9888602786858512

In [34]:
len(pred)

3595872

In [35]:
valid['pred'] = pred

In [36]:
valid_tmp = valid

In [37]:
valid_tmp = valid_tmp[['id', 'match_id', 'label', 'pred']]

In [38]:
# IoUを計算する時は自分自身も加えるのでuniqueなidを計算しておく
unum = len(valid_tmp['id'].unique())
unum

113881

In [39]:
len(valid_tmp.query('label==1'))

96049

In [40]:
len(valid_tmp.query('pred>0.5'))

78987

In [41]:
for THRESHOLD in (0.4, 0.45, 0.50, 0.55):
  print("THRESHOLD=", THRESHOLD, "****************")
  true_num = len(valid_tmp.query('label==1'))
  pred_num = len(valid_tmp.query(f'pred>{THRESHOLD}'))
  a = len(valid_tmp.query(f'label==1 or pred>{THRESHOLD}'))
  b = len(valid_tmp.query(f'label==1 and pred>{THRESHOLD}'))
  print("true_num:", true_num, "pred_num:", pred_num)
  print('a, b=', a, b)
  print('IoU=', (b + unum) / (a + unum))
#print('IoU=', b/a)

THRESHOLD= 0.4 ****************
true_num: 96049 pred_num: 87765
a, b= 111892 71922
IoU= 0.8229637733475659
THRESHOLD= 0.45 ****************
true_num: 96049 pred_num: 83208
a, b= 109424 69833
IoU= 0.822704372942836
THRESHOLD= 0.5 ****************
true_num: 96049 pred_num: 78987
a, b= 107315 67721
IoU= 0.8210003797537025
THRESHOLD= 0.55 ****************
true_num: 96049 pred_num: 74970
a, b= 105520 65499
IoU= 0.8175897101654049
